In [2]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from joblib import dump
print(os.listdir("./"))

['.ipynb_checkpoints', 'AnikaML.ipynb', 'AnikaMLUpdated-forecasting-sarima.ipynb', 'AnikaMLUpdated-forecasting.ipynb', 'AnikaMLUpdated-prediction-baseline.ipynb', 'AnikaMLUpdated-prediction-lgb.ipynb', 'AnikaMLUpdated-prediction-xgb-params.ipynb', 'calendar.csv', 'calendar_events.csv', 'CHAUHAN_ANIKA-14188775-AT2', 'df_cleaned.csv', 'items_weekly_sell_prices.csv', 'melted_df.csv', 'models', 'New folder', 'notebooks', 'reports', 'sales_test.csv', 'sales_train.csv', 'screenshots', 'Untitled.ipynb', 'Untitled1.ipynb']


In [3]:
melted_df=pd.read_csv("melted_df.csv")

In [4]:
len(melted_df)

46985090

In [5]:
data_price_df = pd.read_csv('./items_weekly_sell_prices.csv') 
data_price_df['item_id'] = data_price_df['item_id'].astype(str)

In [6]:
calendar_data = pd.read_csv('./calendar.csv')
len(calendar_data)

1969

In [7]:
train_data = pd.merge(melted_df,calendar_data, left_on='day', right_on='d', how='left')
columns_to_drop = ['d', 'day']
train_data = train_data.drop(columns=columns_to_drop)


# Create label encoders for categorical columns
label_encoders = {}
categorical_cols = ['item_id', 'store_id']
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le
    
# Encode categorical variables using label encoding for prices data
categorical_cols_sp = ['item_id', 'store_id']
for col in categorical_cols_sp:
    le = label_encoders[col]
    data_price_df[col] = le.transform(data_price_df[col])


In [8]:
data_price_df

,store_id,item_id,wm_yr_wk,sell_price
0,0,1437,11325,9.58
1,0,1437,11326,9.58
2,0,1437,11327,8.26
3,0,1437,11328,8.26
4,0,1437,11329,8.26
...,...,...,...,...
6841116,9,1436,11617,1.00
6841117,9,1436,11618,1.00
6841118,9,1436,11619,1.00
6841119,9,1436,11620,1.00


In [9]:
train_data

,item_id,store_id,value,date,wm_yr_wk
0,1437,0,0,2011-01-29,11101
1,1438,0,0,2011-01-29,11101
2,1439,0,0,2011-01-29,11101
3,1440,0,0,2011-01-29,11101
4,1441,0,0,2011-01-29,11101
...,...,...,...,...,...
46985085,1432,9,2,2015-04-18,11512
46985086,1433,9,0,2015-04-18,11512
46985087,1434,9,0,2015-04-18,11512
46985088,1435,9,0,2015-04-18,11512


In [10]:
right_join = pd.merge(train_data,data_price_df, left_on=['store_id','item_id','wm_yr_wk'], right_on=['store_id','item_id','wm_yr_wk'], how='right')
right_join

,item_id,store_id,value,date,wm_yr_wk,sell_price
0,1437,0,0.0,2013-07-13,11325,9.58
1,1437,0,0.0,2013-07-14,11325,9.58
2,1437,0,0.0,2013-07-15,11325,9.58
3,1437,0,0.0,2013-07-16,11325,9.58
4,1437,0,0.0,2013-07-17,11325,9.58
...,...,...,...,...,...,...
36575940,1436,9,NaN,NaN,11617,1.00
36575941,1436,9,NaN,NaN,11618,1.00
36575942,1436,9,NaN,NaN,11619,1.00
36575943,1436,9,NaN,NaN,11620,1.00


In [11]:
df_cleaned = right_join.dropna(subset=['value'])

In [12]:
df_cleaned['cya_target'] = df_cleaned['value'] * df_cleaned['sell_price']

df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], format='%Y-%m-%d')

C:\Users\anika\AppData\Local\Temp\ipykernel_5636\3642098738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cya_target'] = df_cleaned['value'] * df_cleaned['sell_price']
C:\Users\anika\AppData\Local\Temp\ipykernel_5636\3642098738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], format='%Y-%m-%d')


In [13]:
df_cleaned['date_timestamp'] = df_cleaned['date'].values.astype('int64') // 10**9  # Convert to seconds

df_cleaned = df_cleaned.drop(columns=['date'])


C:\Users\anika\AppData\Local\Temp\ipykernel_5636\2260052284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['date_timestamp'] = df_cleaned['date'].values.astype('int64') // 10**9  # Convert to seconds


In [14]:
numerical_cols_int = ['item_id', 'store_id', 'date_timestamp']
for col in numerical_cols_int:
    df_cleaned.loc[:, col] = df_cleaned[col].astype('int64')
numeric_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)


In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols_int)
    ]
)

In [17]:
# Split data into features and target variable
X = df_cleaned[['item_id', 'store_id', 'date_timestamp']]
y = df_cleaned['cya_target']

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Calculate the mean of the target variable 'cya_target' in the training data
baseline_prediction = y_train.mean()

# Create an array of baseline predictions with the same length as the evaluation data
baseline_predictions = [baseline_prediction] * len(y_eval)

# Calculate the Mean Absolute Error (MAE) for the baseline model
mae_baseline = mean_absolute_error(y_eval, baseline_predictions)

print("Mean Absolute Error (MAE) for Baseline Model (Mean Prediction):", mae_baseline)

Mean Absolute Error (MAE) for Baseline Model (Mean Prediction): 5.29756962824539
